In [62]:
!pip3 install keras
!pip3 install xgboost
!pip3 install catboost ipywidgets
!pip3 install tensorflow

In [63]:
import json
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.datasets import make_regression
from sklearn.svm import SVR
import sklearn.linear_model as skl_lm
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline

from catboost import CatBoostRegressor

from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Dense, Dropout, BatchNormalization

In [64]:
with open('repos.json') as json_file:
    repos = json.load(json_file)

In [65]:
df_repos = pd.DataFrame(repos.values())

In [66]:
X = df_repos.drop(['owner', 'name', 'id', 'full_name', 'created_at', 'last_commit', 'main_language', 'stargazer_count'], axis=1)
y = df_repos['stargazer_count']
y = y.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
column_dim = len(X.columns)

In [67]:
#Random forest
rf = RandomForestRegressor()

pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca),
                       ('forest', rf)])

rf.fit(X_train, y_train)

training_score = rf.score(X_train, y_train)
test_score = rf.score(X_test, y_test)

print("training score: ", training_score)
print("test score: ", test_score)

training score:  0.9725095507738531
test score:  0.7900667508555722


In [68]:
feature_importances=pd.DataFrame(rf.feature_importances_,
                                 index=X.columns,
                                 columns=['importance']).sort_values('importance',ascending=False)

In [12]:
#Gradient boost

reg = GradientBoostingRegressor(verbose = 1, n_estimators = 100, learning_rate = 0.25)
reg.fit(X_train, y_train)

training_score = reg.score(X_train, y_train)
test_score = reg.score(X_test, y_test)

print("training score: ", training_score)
print("test score: ", test_score)

      Iter       Train Loss   Remaining Time 
         1    40179783.7243            3.67s
         2    27533942.8212            3.58s
         3    19851292.5602            3.46s
         4    15350633.3344            3.38s
         5    12592133.5885            3.33s
         6    10534455.8922            3.31s
         7     9255505.0444            3.26s
         8     8448094.5089            3.21s
         9     7802364.3303            3.17s
        10     7325814.2061            3.15s
        20     5271880.4988            2.80s
        30     4414368.7585            2.43s
        40     3804142.7725            2.08s
        50     3378611.6085            1.73s
        60     2999456.8267            1.39s
        70     2778809.2491            1.04s
        80     2529855.5396            0.69s
        90     2366885.4461            0.35s
       100     2255304.3751            0.00s
training score:  0.9630602766245615
test score:  0.788918307985317


In [13]:
# Neural Network
X_train = np.asarray(X_train).astype('float32')
def baseline_model():
    model = Sequential()
    model.add(Dense(100,input_dim=column_dim, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))    
    model.add(Dense(80, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(60, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))   
    model.add(Dense(40, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))
    #model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal'))
    #model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='glorot_normal'))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=200, epochs=100, batch_size=32)

estimator.fit(X_train, y_train)

Epoch 1/100
282/282 [==============================] - 17s 3ms/step - loss: 6983295634.8269
Epoch 2/100
282/282 [==============================] - 1s 2ms/step - loss: 1343883248.3675
Epoch 3/100
282/282 [==============================] - 1s 2ms/step - loss: 7092841742.4735
Epoch 4/100
282/282 [==============================] - 1s 2ms/step - loss: 498307645.6820
Epoch 5/100
282/282 [==============================] - 1s 3ms/step - loss: 576623706.4594
Epoch 6/100
282/282 [==============================] - 1s 2ms/step - loss: 267267812.9170
Epoch 7/100
282/282 [==============================] - 1s 2ms/step - loss: 522915924.1837
Epoch 8/100
282/282 [==============================] - 1s 2ms/step - loss: 737979061.4240
Epoch 9/100
282/282 [==============================] - 1s 2ms/step - loss: 326433178.1201
Epoch 10/100
282/282 [==============================] - 1s 2ms/step - loss: 113522953.7456
Epoch 11/100
282/282 [==============================] - 1s 2ms/step - loss: 309477445.4770
Epoc

In [14]:
X_test = np.asarray(X_test).astype('float32')

train_pred = estimator.predict(X_train)
test_pred = estimator.predict(X_test)

training_score = r2_score(y_train, train_pred)
test_score = r2_score(y_test, test_pred)

print("training score: ", training_score)
print("test score: ", test_score)

training score:  0.8009505697207137
test score:  0.7569913331111832


In [15]:
# k-NN
model=KNeighborsRegressor(n_neighbors=2)
model.fit(X_train,y_train)

train_pred=model.predict(X_train)
test_pred=model.predict(X_test)

training_score = r2_score(y_train, train_pred)
test_score = r2_score(y_test, test_pred)

print("training score: ", training_score)
print("test score: ", test_score)

training score:  0.7654478922792048
test score:  0.30660919365378037


In [16]:
# catboost

modelCat = CatBoostRegressor(iterations= 440, depth= 8,
                             learning_rate= 0.1, loss_function='RMSE')
#modelCat.fit(X_train[:90503], y_train[:90503] , eval_set=(X_train[90503:], y_train[90503:]),plot=True)
modelCat.fit(X_train, y_train)

train_pred  =  modelCat.predict(X_train)
pred = modelCat.predict(X_test)

train_score = r2_score(y_train , train_pred)
test_score = r2_score(y_test, pred)

print("Training score: ", train_score)
print("Test score: ", test_score)

0:	learn: 7354.4056119	total: 70.8ms	remaining: 31.1s
1:	learn: 6911.0822408	total: 85.7ms	remaining: 18.8s
2:	learn: 6486.6484622	total: 99.6ms	remaining: 14.5s
3:	learn: 6095.7606037	total: 115ms	remaining: 12.5s
4:	learn: 5772.6539608	total: 130ms	remaining: 11.3s
5:	learn: 5486.5082259	total: 144ms	remaining: 10.4s
6:	learn: 5212.2858801	total: 162ms	remaining: 10s
7:	learn: 4970.8954915	total: 177ms	remaining: 9.57s
8:	learn: 4772.1008659	total: 192ms	remaining: 9.18s
9:	learn: 4574.5796792	total: 206ms	remaining: 8.85s
10:	learn: 4376.2252051	total: 220ms	remaining: 8.57s
11:	learn: 4212.3953106	total: 234ms	remaining: 8.36s
12:	learn: 4057.3384806	total: 250ms	remaining: 8.22s
13:	learn: 3922.6538666	total: 268ms	remaining: 8.14s
14:	learn: 3811.0318625	total: 282ms	remaining: 7.98s
15:	learn: 3690.9688201	total: 297ms	remaining: 7.86s
16:	learn: 3577.9872449	total: 311ms	remaining: 7.75s
17:	learn: 3471.9228895	total: 325ms	remaining: 7.62s
18:	learn: 3383.3703139	total: 339ms	